# マイクロマウスのターン生成

## 概要
マイクロマウス競技において、高速な走行を行うためにはスラロームと呼ばれる車体を止めずにターンをする動作が必要とされます。
このプログラムでは、スラローム走行に必要な軌跡を生成します。

## インプット
軌跡を作る際のインプットは以下になります。
    ターンの始点、終点、ターン角度、初期角度、曲線長、角速度変化調整用変数

以下のように直線と曲線を組み合わせてターンを生成します。

    直線 -> 曲線 -> 直線

曲線前後の直線の長さは曲線の長さに応じて自動で決定します。    

## アウトプット
スラロームの軌跡は弧長パラメータ付けされた曲率のリストで出力されます。
ロボットに実装する際には曲率のリストをロボットのプログラムにコピペし、軌道計画に利用してください。
軌道計画の方法についてはREADME.ipynbが参考になるかもしれません。

## マイクロマウス競技で用いるターン

### スラローム レベル 1 
まず、最低限のスラロームとして以下を実装しましょう。

 - 探索90度ターン
 - 最短90度ターン

### スラローム レベル 2
最短走行の際に半径の大きいターンを行うことでより高速で走行が可能です。

 - 大周り90度ターン
 - 180度ターン

### スラローム レベル 3
最短走行の際に区画を斜めに進むことで大幅な走行時間短縮が可能です。
ただし、壁にぶつからないように制御を行うことが少々難しいです。

 - 直進から斜め45度ターン
 - 斜めから直進45度ターン
 - 直進から斜め135度ターン
 - 斜めから直進135度ターン
 - 斜め90度ターン

このうち、45度ターンと135度ターンは直進から斜めと斜めから直進のパターンがありますが、軌跡を逆からたどれば一方が他方になるので、実質3つのターンになります。
### スラローム レベル 4
上記以外にも変則的なスラローム起動が存在します。
(こじまターン等と呼ばれるらしいです。)
    

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt 
import matplotlib.ticker as tick
import numpy as np 
from numba import jit
import math 
from itertools import product

from ipywidgets import interact, interactive, fixed




In [2]:
# molifireの積分値を計算するための関数


#y = math.e / a * np.exp( (-1)/(1- np.abs(2*x/b -1)**c))

@jit
def moli(x,a,b,c):
    return math.e / a * np.exp( (-1)/(1- np.abs(2*x/b -1)**c))

@jit
def int_moli(c):
    delta_t = 0.00001
    x = np.arange(0.000001,1,delta_t)

    S = 0
    y = moli(x,1,1,c)
    for i,ele in enumerate(y):
        if i != 0:
            S = S + delta_t * ele
    return S    


In [3]:

# 軌跡を作るためのクラス
class CalcTraj:
    shape_factor = 0
    path_length = 0
    target_ang = 0
    delta_t = 0
    tread = 0
    start_ang = 0
    start_x = 0
    start_y = 0
    
    s = None
    kappa =  None
    D_kappa = None
    I_kappa = None
    c_traj_x = None
    c_traj_y = None
    l_traj_x = None
    l_traj_y = None
    r_traj_x = None
    r_traj_y = None
    
    # 軌跡のパラメータをセットする
    def set_param(self, shape_factor_, path_length_, target_ang_, delta_t_, tread_, start_x_=0, start_y_=0, start_ang_=0):
        self.shape_factor = shape_factor_
        self.path_length = path_length_
        self.target_ang = target_ang_
        self.delta_t = delta_t_
        self.tread = tread_
        self.start_x = start_x_
        self.start_y = start_y_
        self.start_ang = start_ang_
        
    # 軌跡を計算する    
    @jit
    def calc_traj(self):
        self.s = np.arange(0.00001, self.path_length, self.delta_t)
        
        # a * b * S = target_ang
        c = self.shape_factor
        b = self.path_length
        S = int_moli(c)
        a = 1/(self.target_ang/b/S)
        
        # 曲率 
        self.kappa = np.zeros(len(self.s))
        for i in range(len(self.s)):
            val = 1 - np.abs(2*self.s[i]/b -1)**c
            if val != 0.0:
                self.kappa[i] = math.e / a * np.exp(-1/val) 
            else:
                self.kappa[i] = 0.0
        
        # 曲率の微分
        self.D_kappa = np.gradient(self.kappa)/ self.delta_t

        
        # 曲率の積分
        self.I_kappa = np.zeros(len(self.kappa))
        self.I_kappa[0] = self.start_ang;
        for i,ele in enumerate(self.kappa):
            if i != 0:
                self.I_kappa[i] = self.I_kappa[i-1] + self.delta_t * ele 

        # 軌跡のx座標, y座標を求める        
        self.c_traj_x = np.zeros(len(self.kappa))
        self.c_traj_y = np.zeros(len(self.kappa))

        self.l_traj_x = np.zeros(len(self.kappa))
        self.l_traj_y = np.zeros(len(self.kappa))

        self.r_traj_x = np.zeros(len(self.kappa))
        self.r_traj_y = np.zeros(len(self.kappa))

        self.c_traj_x[0] = self.start_x
        self.c_traj_y[0] = self.start_y
        
        self.r_traj_x[0] = self.c_traj_x[0] + math.sin(self.start_ang)*self.tread/2
        self.r_traj_y[0] = self.c_traj_y[0] - math.cos(self.start_ang)*self.tread/2

        self.l_traj_x[0] = self.c_traj_x[0] - math.sin(self.start_ang)*self.tread/2
        self.l_traj_y[0] = self.c_traj_y[0] + math.cos(self.start_ang)*self.tread/2
        
        for i,ele in enumerate(self.I_kappa):
            if i != 0:
                self.c_traj_x[i] = self.c_traj_x[i-1] + self.delta_t *  math.cos(ele)
                self.c_traj_y[i] = self.c_traj_y[i-1] + self.delta_t *  math.sin(ele)
                self.r_traj_x[i] = self.c_traj_x[i] + math.sin(ele)*self.tread/2
                self.r_traj_y[i] = self.c_traj_y[i] - math.cos(ele)*self.tread/2

                self.l_traj_x[i] = self.c_traj_x[i] - math.sin(ele)*self.tread/2
                self.l_traj_y[i] = self.c_traj_y[i] + math.cos(ele)*self.tread/2
        
            
    # 軌跡が終わったポイントのx座標
    def get_end_x(self):
        return self.c_traj_x[len(self.c_traj_x)-1]
    # 軌跡が終わったポイントのy座標    
    def get_end_y(self):
        return self.c_traj_y[len(self.c_traj_y)-1]
    
    # x方向の移動距離
    def get_move_x(self):
        return self.c_traj_x[len(self.c_traj_x)-1] - self.start_x
    
    # y方向の移動距離
    def get_move_y(self):
        return self.c_traj_y[len(self.c_traj_y)-1] - self.start_y
        
    # 速度一定で走ったときの時間と角速度のリストを返す
    def get_omega(self, v):
        return self.s /v, self.kappa * v
    
    # 速度一定で走ったときの時間と角速度(degree)のリストを返す
    def get_omega_degree(self, v):
        t, rad = self.get_omega(v) 
        return t, rad * 180 /math.pi
    
    # 速度一定で走ったときの時間と角加速度のリストを返す
    def get_alpha(self, v):
        t, rad = self.get_omega(v)
        alpha = np.gradient(rad)/ (self.delta_t/v)
        return t,alpha 

    # 速度一定で走ったときの時間と角加速度(degree)のリストを返す
    def get_alpha_degree(self, v):
        t, alpha = self.get_alpha(v)
        return t, alpha * 180/math.pi
    
    # 速度一定で走ったときの時間と向進加速度のリストを返す
    def get_cent_acc(self, v):
        return self.s/v, v * v * self.kappa 
        
    
    
    # 速度一定で走ったときに右タイヤにかかる力を返す
    def get_r_force(self, v, m, I):
        t, rad = self.get_omega(v)
        t, alpha = self.get_alpha(v)
        omega_f = m * v * 0.5 * rad
        alpha_f = I * alpha / (2 * self.tread)
        f = np.sqrt(omega_f * omega_f + alpha_f * alpha_f)
        return t, omega_f, alpha_f, f
    
    # 速度一定で走ったときに左タイヤにかかる力を返す
    def get_l_force(self, v, m, I):
        t, rad = self.get_omega(v)
        t, alpha = self.get_alpha(v)
        omega_f = m * v * 0.5 * rad
        alpha_f = I * alpha / (2 * self.tread)
        f =  np.sqrt(omega_f * omega_f + alpha_f * alpha_f)
        return t, omega_f, alpha_f, f
        

## クラシック探索90度ターン
壁を読む関係上ターンの開始、終了を1cmずらしています。

In [11]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.01 -0.09
start_y = 0.00
end_x = 0.0
end_y = -0.01 + 0.09
target_ang = 90
start_ang = 0.0

shape_factor__ = 2.6
path_length__ = 0.09
v__ = 1.0
traj_cl_serch_90 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_cl_serch_90
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_pre = (end_x - start_x) - traj_curve.get_move_x()
    d_fol = (end_y - start_y) - traj_curve.get_move_y()
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.get_end_x()+ d_pre +tread/2 , end_x+tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre -tread/2, end_x-tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 

    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.title("trajectory")
    plt.axis([-0.36, 0.36, -0.36, 0.36])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.18+ 0.09-0.006, j*0.18+0.09-0.006], width=0.012, height=0.012) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()
    

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [ ]:
print(" ")
print("len(kappa) " + str(len(traj_cl_serch_90.kappa)))
print("s resolution[m] " + str(traj_cl_serch_90.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_cl_serch_90.s, traj_cl_serch_90.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

## クラシック最短90度

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.09
start_y = 0.00
end_x = 0.0
end_y = 0.09
target_ang = 90
start_ang = 0.0

shape_factor__ = 2.6
path_length__ = 0.09
v__ = 1.0
traj_cl_90 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_cl_90
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_pre = (end_x - start_x) - traj_curve.get_move_x()
    d_fol = (end_y - start_y) - traj_curve.get_move_y()
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.get_end_x()+ d_pre +tread/2 , end_x+tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre -tread/2, end_x-tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 

    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.title("trajectory")
    plt.axis([-0.36, 0.36, -0.36, 0.36])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.18+ 0.09-0.006, j*0.18+0.09-0.006], width=0.012, height=0.012) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()
    

In [ ]:
print(" ")
print("len(kappa) " + str(len(traj_cl_90.kappa)))
print("s resolution[m] " + str(traj_cl_90.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_cl_90.s, traj_cl_90.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

## クラシック大周り90度

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.18
start_y = 0.00
end_x = 0.0
end_y = 0.18
target_ang = 90
start_ang = 0.0

shape_factor__ = 2.6
path_length__ = 0.2
v__ = 1.0
traj_cl_L_90 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_cl_L_90
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_pre = (end_x - start_x) - traj_curve.get_move_x()
    d_fol = (end_y - start_y) - traj_curve.get_move_y()
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.get_end_x()+ d_pre +tread/2 , end_x+tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre -tread/2, end_x-tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 

    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.title("trajectory")
    plt.axis([-0.36, 0.36, -0.36, 0.36])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.18+ 0.09-0.006, j*0.18+0.09-0.006], width=0.012, height=0.012) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()
    

In [ ]:
print(" ")
print("len(kappa) " + str(len(traj_cl_90.kappa)))
print("s resolution[m] " + str(traj_cl_90.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_cl_L_90.s, traj_cl_L_90.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

## クラシック180度ターン
180度ターンは曲線長と角度を指定するだけでうまく終点と始点が結べないため注意。
カーブの終点が180度ターンの終了点になるように曲線長を2分探索で求めている。

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.18
start_y = 0.00
end_x = -0.18
end_y = 0.18
target_ang = 180
start_ang = 0.0

shape_factor__ = 2.6

v__ = 1.0
traj_cl_180 = CalcTraj()
d_pre__ = 0.01

@interact(shape_factor=(0.1, 8.0, 0.1), d_pre=(0.001, 0.3, 0.001) ,  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, d_pre = d_pre__, v=v__):
    
    traj_curve = traj_cl_180
    path_length_min = 0.1
    path_length_max = 1.0
    error_y = 1.0
    
    # 最小化するターゲット関数
    def calc_y_error(pl_):
        traj = CalcTraj()
        traj.set_param(shape_factor_=shape_factor, path_length_= pl_, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)        
        traj.calc_traj()
        error_y_ = traj.get_move_y() - end_y
        return error_y_
    
    while abs(error_y) > 0.0001:
        pl = 0.5 * (path_length_min + path_length_max)
        error_y = calc_y_error(pl)
        path_length = pl
        if(error_y > 0):
            path_length_max = pl
        else:
            path_length_min = pl
        print("error_y: " + str(error_y))
        print("path_length: " + str(pl))
        print("min, max :",path_length_min, path_length_max)
        
    d_fol = d_pre
    
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)        
    traj_curve.calc_traj()
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y()+tread/2, traj_curve.get_end_y()+tread/2]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), traj_curve.get_end_y()]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y()-tread/2, traj_curve.get_end_y()-tread/2]  ,color="red") 


    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.title("trajectory")
    plt.axis([-0.36, 0.36, -0.36, 0.36])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.18+ 0.09-0.006, j*0.18+0.09-0.006], width=0.012, height=0.012) )
    plt.show()

    print("move_y[m] " + str(traj_curve.get_move_y()) )  
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()
    

In [ ]:
print(" ")
print("len(kappa) " + str(len(traj_cl_180.kappa)))
print("s resolution[m] " + str(traj_cl_180.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_cl_180.s, traj_cl_180.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

## クラシック 斜め 45度ターン

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.18
start_y = 0.00
end_x = 0.0
end_y = 0.09
target_ang = 45
start_ang = 0.0

shape_factor__ = 2.6
path_length__ = 0.12
v__ = 1.0
traj_cl_45 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_cl_45
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_fol = ((end_y - start_y) - traj_curve.get_move_y()) * math.sqrt(2)
    d_pre = (end_x - start_x) - traj_curve.get_move_x() - d_fol/math.sqrt(2)
    
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.r_traj_x[-1] + d_pre , traj_curve.r_traj_x[-1] + d_pre + d_fol/math.sqrt(2) ],[traj_curve.r_traj_y[-1], traj_curve.r_traj_y[-1]+ d_fol/math.sqrt(2)]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.l_traj_x[-1] + d_pre , traj_curve.l_traj_x[-1] + d_pre + d_fol/math.sqrt(2) ],[traj_curve.l_traj_y[-1], traj_curve.l_traj_y[-1]+ d_fol/math.sqrt(2)]  ,color="red") 

    
    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.title("trajectory")
    plt.axis([-0.36, 0.36, -0.36, 0.36])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.18+ 0.09-0.006, j*0.18+0.09-0.006], width=0.012, height=0.012) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()
    

In [ ]:
print(" ")
print("len(kappa) " + str(len(traj_cl_45.kappa)))
print("s resolution[m] " + str(traj_cl_45.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_cl_45.s, traj_cl_45.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

## クラシック 斜め 135度ターン

In [4]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.18
start_y = 0.00
end_x = -0.09
end_y = 0.18
target_ang = 135
start_ang = 0.0

shape_factor__ = 2.6
path_length__ = 0.25
v__ = 1.0
traj_cl_135 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_cl_135
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_fol = ((end_y - start_y) - traj_curve.get_move_y()) * math.sqrt(2)
    d_pre = (end_x - start_x) - traj_curve.get_move_x() + d_fol/math.sqrt(2)
    
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.r_traj_x[-1] + d_pre , traj_curve.r_traj_x[-1] + d_pre - d_fol/math.sqrt(2) ],[traj_curve.r_traj_y[-1], traj_curve.r_traj_y[-1]+ d_fol/math.sqrt(2)]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.l_traj_x[-1] + d_pre , traj_curve.l_traj_x[-1] + d_pre - d_fol/math.sqrt(2) ],[traj_curve.l_traj_y[-1], traj_curve.l_traj_y[-1]+ d_fol/math.sqrt(2)]  ,color="red") 

    
    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.title("trajectory")
    plt.axis([-0.36, 0.36, -0.36, 0.36])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.18+ 0.09-0.006, j*0.18+0.09-0.006], width=0.012, height=0.012) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [ ]:
print(" ")
print("len(kappa) " + str(len(traj_cl_135.kappa)))
print("s resolution[m] " + str(traj_cl_135.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_cl_135.s, traj_cl_135.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

## クラシック 斜め 90度ターン

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.09
start_y = 0.00
end_x = -0.09
end_y = 0.18
target_ang = 90
start_ang = 45

shape_factor__ = 2.6
path_length__ = 0.15
v__ = 1.0
traj_cl_D_90 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_cl_D_90
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_fol = ((end_y - start_y) - traj_curve.get_move_y()) /(math.sqrt(2))
    d_pre = d_fol
    
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    #plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([traj_curve.r_traj_x[0]  ,traj_curve.r_traj_x[0] + d_pre/math.sqrt(2)],[traj_curve.r_traj_y[0], traj_curve.r_traj_y[0] + d_pre/math.sqrt(2)]  ,color="red") 
    plt.plot([start_x,traj_curve.c_traj_x[0] + d_pre/math.sqrt(2)],[start_y, traj_curve.c_traj_y[0] + d_pre/math.sqrt(2)]  ,color="red") 
    plt.plot([traj_curve.l_traj_x[0], traj_curve.l_traj_x[0] + d_pre/math.sqrt(2)],[traj_curve.l_traj_y[0], traj_curve.l_traj_y[0] + d_pre/math.sqrt(2)]  ,color="red") 

    #plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.r_traj_x + d_pre/math.sqrt(2), traj_curve.r_traj_y + d_pre/math.sqrt(2), color="blue")
    plt.plot(traj_curve.c_traj_x + d_pre/math.sqrt(2), traj_curve.c_traj_y + d_pre/math.sqrt(2) , color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre/math.sqrt(2), traj_curve.l_traj_y + d_pre/math.sqrt(2), color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.r_traj_x[-1] + d_pre/math.sqrt(2) , traj_curve.r_traj_x[-1]  ],[traj_curve.r_traj_y[-1]+ d_pre/math.sqrt(2), traj_curve.r_traj_y[-1]+ 2*d_fol/math.sqrt(2)]  ,color="red") 
    plt.plot([traj_curve.get_end_x() + d_pre/math.sqrt(2) , end_x],[traj_curve.get_end_y() + d_pre/math.sqrt(2), end_y]  ,color="red") 
    plt.plot([traj_curve.l_traj_x[-1] + d_pre/math.sqrt(2) , traj_curve.l_traj_x[-1]  ],[traj_curve.l_traj_y[-1]+ d_pre/math.sqrt(2) , traj_curve.l_traj_y[-1]+ 2*d_fol/math.sqrt(2)]  ,color="red") 

    
    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.title("trajectory")
    plt.axis([-0.36, 0.36, -0.36, 0.36])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.18+ 0.09-0.006, j*0.18+0.09-0.006], width=0.012, height=0.012) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()

In [ ]:
print(" ")
print("len(kappa) " + str(len(traj_cl_D_90.kappa)))
print("s resolution[m] " + str(traj_cl_D_90.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_cl_D_90.s, traj_cl_D_90.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )